# Projet

## Prérequis

In [26]:
# Modules standards
import sys
import os
import importlib

# Définir les chemins du projet
PROJECT_PATH = os.getcwd()
SRC_PATH = os.path.join(PROJECT_PATH, "src")

# Ajouter le dossier src/ au chemin d'import
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

# Vérification des chemins
print("Chemin du projet :", PROJECT_PATH)
print("Chemin du dossier src :", SRC_PATH)

# Importation des modules personnalisés
import extract_data
import explore_data

from extract_data import *
from explore_data import *

# Rechargement si modification en cours de session
importlib.reload(extract_data)
importlib.reload(explore_data)

Chemin du projet : c:\Users\marct\Documents\GIF-7005\Projet
Chemin du dossier src : c:\Users\marct\Documents\GIF-7005\Projet\src


<module 'explore_data' from 'c:\\Users\\marct\\Documents\\GIF-7005\\Projet\\src\\explore_data.py'>

## Importation des données

In [23]:
csv_path = os.path.join(PROJECT_PATH, "data", "raw", "FM12", "red", "train_12.csv")
data = get_data(csv_path)

c:\Users\marct\Documents\GIF-7005\Projet\src\extract_data.py:5: DtypeWarning: Columns (4,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


## Exploration des données

In [ ]:
from explore_data import *

save_path = os.path.join(PROJECT_PATH, "outputs", "exploration", "rapport_FM12.html")

explore_colnames = [
    "Credit_Score", "Mortgage_Insurance", "Number_of_units",
    "CLoan_to_value", "Debt_to_income", "OLoan_to_value",
    "Single_borrower",
    "is_Loan_purpose_purc", "is_Loan_purpose_cash", "is_Loan_purpose_noca",
    "is_First_time_homeowner", "is_First_time_homeowner_No",
    "is_Occupancy_status_prim", "is_Occupancy_status_inve", "is_Occupancy_status_seco",
    "is_Origination_channel_reta", "is_Origination_channel_brok", "is_Origination_channel_corr", "is_Origination_channel_tpo",
    "is_Property_type_cond", "is_Property_type_coop", "is_Property_type_manu",
    "is_Property_type_pud", "is_Property_type_sing",
    "DFlag"
]

data_subset = data[explore_colnames]
summarize_data_to_html(data_subset, "FM12 - Rapport", save_path)

c:\Users\marct\Programs\anaconda\envs\gif7005-env\Lib\site-packages\ydata_profiling\utils\dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 167.89it/s]


## Prétraitement des données